# Code for Capstone Project

## Load Necessary Libraries

In [18]:
import json # library to handle JSON files
import pandas as pd

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

In [3]:
import pandas as pd
#Load the list of cities table in BC
Metrovan_wiki= pd.read_html('https://en.wikipedia.org/wiki/List_of_cities_in_British_Columbia')
Metrovan_cities = Metrovan_wiki[0]
#Metrovan_cities

In [4]:
#Processing the data and remove the columns we don't need
Metrovan_df = Metrovan_cities.drop(columns=['Corporate name[2]','Incorporation date[2]','Population (2011)[5]', 'Change (%)[5]', 'Area (km2)[5]'])
Metrovan_df.head()

,Name,Regional district[2],Population (2016)[5],Population density[5]
0,Abbotsford,Fraser Valley,141397,376.5
1,Armstrong,North Okanagan,5114,979.8
2,Burnaby,Metro Vancouver,232755,2568.7
3,Campbell River,Strathcona,32588,225.7
4,Castlegar,Central Kootenay,8039,408.6


In [5]:
#Removing the rows that are now in Metro Vancouver
Metrovan_index = Metrovan_df[Metrovan_df['Regional district[2]']!='Metro Vancouver'].index
Metrovan_df.drop(Metrovan_index, inplace=True)
#Metrovan_df.head()

In [6]:
#Once all the data have been cleaned up, obtain the longitude and latitude data of each city in Metro Vancouver. This is done manually through searching on longlat.net
Latitude = ['49.267132','49.283764','49.084721','49.074329','49.216667','49.206944','49.316666','49.233334','49.262501','49.283054','49.166592','49.104431','49.246292','49.019917']
Longitude = ['-122.968941','-122.793205','-123.058609','-122.559319','-122.599998','-122.911110','-123.066666','-122.683334','-122.781113','-122.831665','-123.133568','-122.801094','-123.116226','-122.802612']

In [7]:
#Combining the Longitude and Latitude Information with the Metro Vancouver Data Frame
Metrovan_df['Latitude'] = Latitude
Metrovan_df['Longitude'] = Longitude
Metrovan_df

,Name,Regional district[2],Population (2016)[5],Population density[5],Latitude,Longitude
2,Burnaby,Metro Vancouver,232755,2568.7,49.267132,-122.968941
7,Coquitlam,Metro Vancouver,139284,1138.9,49.283764,-122.793205
11,Delta,Metro Vancouver,102238,567.4,49.084721,-123.058609
22,Langley,Metro Vancouver,25888,2533.6,49.074329,-122.559319
23,Maple Ridge,Metro Vancouver,82256,308.3,49.216667,-122.599998
27,New Westminster,Metro Vancouver,70996,4543.4,49.206944,-122.911110
28,North Vancouver,Metro Vancouver,52898,4465.1,49.316666,-123.066666
31,Pitt Meadows,Metro Vancouver,18573,214.7,49.233334,-122.683334
33,Port Coquitlam,Metro Vancouver,58612,2009.4,49.262501,-122.781113
34,Port Moody,Metro Vancouver,33551,1295.9,49.283054,-122.831665


## Using Foursquare API to Work With Our Data

In [8]:
##Foursqaure API Information
CLIENT_ID = '54XFAUMF1MA1ZUW035QFJAULEIUHDX05QFJVBCPEE1X1EZ5N'
CLIENT_SECRET = '0110YF5V3MBSUFZQ4W1A5OZQVFS4AFO1OEANCEI2GKCC00AC'
VERSION = '20200701'

In [9]:
#city_latitude = Metrovan_df['Latitude'] # City latitude value
#city_longitude = Metrovan_df['Longitude'] # City longitude value

#city_name = Metrovan_df['Regional district[2]'] # Name
#city_state = BC

#print('Latitude and longitude values of {}, {} are: {}, {}.'.format(city_name,
                                                               #city_state,
                                                               #city_latitude, 
                                                   #city_longitude))
Metrovan_df.iloc[0]

Name                             Burnaby
Regional district[2]     Metro Vancouver
Population (2016)[5]              232755
Population density[5]             2568.7
Latitude                       49.267132
Longitude                    -122.968941
Name: 2, dtype: object

In [24]:
neighborhood_latitude=49.267132
neighborhood_longitude=-122.968941

LIMIT = 10000
radius = 50000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [25]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1fa858d8d6542aaad4a884'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Burnaby',
  'headerFullLocation': 'Burnaby',
  'headerLocationGranularity': 'city',
  'totalResults': 241,
  'suggestedBounds': {'ne': {'lat': 49.71713245000045,
    'lng': -122.28060783171604},
   'sw': {'lat': 48.817131549999544, 'lng': -123.65727416828396}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b4a6ebcf964a5206f8726e3',
       'name': 'Burnaby Mountain Park',
       'location': {'address': '100-300 Centennial Way',
        'crossStreet': 'nr Burnaby Mountain Pkwy',
        'lat': 49.28214015975995,
        'lng': -122.9351234436035,
        'labeledLatLngs

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                   'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
Metrovan_df.head

<bound method NDFrame.head of                Name Regional district[2]  Population (2016)[5]  \
2           Burnaby      Metro Vancouver                232755   
7         Coquitlam      Metro Vancouver                139284   
11            Delta      Metro Vancouver                102238   
22          Langley      Metro Vancouver                 25888   
23      Maple Ridge      Metro Vancouver                 82256   
27  New Westminster      Metro Vancouver                 70996   
28  North Vancouver      Metro Vancouver                 52898   
31     Pitt Meadows      Metro Vancouver                 18573   
33   Port Coquitlam      Metro Vancouver                 58612   
34       Port Moody      Metro Vancouver                 33551   
40         Richmond      Metro Vancouver                198309   
43           Surrey      Metro Vancouver                517887   
46     Vancouver[a]      Metro Vancouver                631486   
50       White Rock      Metro Vancouver      

In [27]:
Vancouver_venues = getNearbyVenues(names=Metrovan_df['Name'],
                                   latitudes=Metrovan_df['Latitude'],
                                   longitudes=Metrovan_df['Longitude']
                                  )
Vancouver_venues

Burnaby
Coquitlam
Delta
Langley
Maple Ridge
New Westminster
North Vancouver
Pitt Meadows
Port Coquitlam
Port Moody
Richmond
Surrey
Vancouver[a]
White Rock


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Burnaby,49.267132,-122.968941,Sarlo's Awesome Eatery,49.266429,-122.971001,Diner
1,Burnaby,49.267132,-122.968941,Sumas Park,49.265564,-122.969760,Park
2,Burnaby,49.267132,-122.968941,Halifax Park,49.268722,-122.962963,Playground
3,Coquitlam,49.283764,-122.793205,Evergreen Cultural Centre,49.285535,-122.790402,Art Gallery
4,Coquitlam,49.283764,-122.793205,City Centre Aquatic Complex,49.285570,-122.794140,Pool
5,Coquitlam,49.283764,-122.793205,Lafarge Lake,49.285767,-122.787858,Lake
6,Coquitlam,49.283764,-122.793205,Urban Gate Bar & Grill,49.281712,-122.796175,Bar
7,Coquitlam,49.283764,-122.793205,The Coffee Bun,49.279554,-122.793293,Café
8,Coquitlam,49.283764,-122.793205,Clever Cupcakes,49.280934,-122.795730,Cupcake Shop
9,Coquitlam,49.283764,-122.793205,Kulinarya,49.282102,-122.799525,Filipino Restaurant


## Use of One Hot Encoding Analyze Each City

In [30]:
Vancouver_onehot = pd.get_dummies(Vancouver_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Vancouver_onehot['Neighborhood'] = Vancouver_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Vancouver_onehot.columns[-1]] + list(Vancouver_onehot.columns[:-1])
Vancouver_onehot = Vancouver_onehot[fixed_columns]

Vancouver_onehot.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,...,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Train Station,Vietnamese Restaurant,Yoga Studio
0,Burnaby,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Burnaby,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Burnaby,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Coquitlam,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Coquitlam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
Vancouver_grouped = Vancouver_onehot.groupby('Neighborhood').mean().reset_index()
Vancouver_grouped.head()

,Neighborhood,American Restaurant,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beach,...,Sushi Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Train Station,Vietnamese Restaurant,Yoga Studio
0,Burnaby,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,Coquitlam,0.0,0.058824,0.058824,0.00,0.000000,0.000000,0.000000,0.058824,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.058824,0.0
2,Delta,0.0,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,Langley,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,Maple Ridge,0.0,0.000000,0.000000,0.00,0.032258,0.032258,0.032258,0.000000,0.0,...,0.064516,0.0,0.0,0.0,0.032258,0.032258,0.0,0.0,0.000000,0.0


In [33]:
num_top_venues = 5

for hood in Vancouver_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Vancouver_grouped[Vancouver_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Burnaby----
                 venue  freq
0           Playground  0.33
1                Diner  0.33
2                 Park  0.33
3  American Restaurant  0.00
4                Hotel  0.00


----Coquitlam----
                    venue  freq
0            Dessert Shop  0.06
1                    Café  0.06
2             Art Gallery  0.06
3               Gastropub  0.06
4  Furniture / Home Store  0.06


----Delta----
                 venue  freq
0          Bus Station  0.25
1   Athletics & Sports  0.25
2                  Gym  0.25
3          Coffee Shop  0.25
4  American Restaurant  0.00


----Langley----
                 venue  freq
0                Diner   1.0
1  American Restaurant   0.0
2                Hotel   0.0
3           Playground   0.0
4          Pizza Place   0.0


----Maple Ridge----
                venue  freq
0          Restaurant  0.10
1  Chinese Restaurant  0.10
2    Sushi Restaurant  0.06
3         Coffee Shop  0.06
4       Grocery Store  0.06


----New Westminster----


In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Vancouver_grouped['Neighborhood']

for ind in np.arange(Vancouver_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Vancouver_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Burnaby,Playground,Park,Diner,Gym,Cosmetics Shop,Cupcake Shop,Department Store,Dessert Shop,Electronics Store,Falafel Restaurant
1,Coquitlam,Dessert Shop,Filipino Restaurant,Chinese Restaurant,Coffee Shop,Café,Pool,Lake,Bubble Tea Shop,Korean Restaurant,Clothing Store
2,Delta,Gym,Coffee Shop,Athletics & Sports,Bus Station,Fish & Chips Shop,Department Store,Dessert Shop,Diner,Electronics Store,Falafel Restaurant
3,Langley,Diner,Yoga Studio,Fish & Chips Shop,Cupcake Shop,Department Store,Dessert Shop,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant
4,Maple Ridge,Chinese Restaurant,Restaurant,Coffee Shop,Grocery Store,Sushi Restaurant,Pharmacy,Burger Joint,Nightlife Spot,Diner,Furniture / Home Store
5,New Westminster,Coffee Shop,Comedy Club,Pub,Historic Site,Sandwich Place,Gym,Japanese Restaurant,Mexican Restaurant,Other Great Outdoors,Cocktail Bar
6,North Vancouver,Gym / Fitness Center,Café,Chinese Restaurant,Vietnamese Restaurant,Greek Restaurant,Fast Food Restaurant,Department Store,Dessert Shop,Diner,Electronics Store
7,Pitt Meadows,Hotel,Liquor Store,Pub,Fish & Chips Shop,Cupcake Shop,Department Store,Dessert Shop,Diner,Electronics Store,Falafel Restaurant
8,Port Coquitlam,Coffee Shop,Sandwich Place,Ice Cream Shop,Bakery,Pizza Place,American Restaurant,Gym / Fitness Center,Falafel Restaurant,Japanese Restaurant,Convenience Store
9,Port Moody,Coffee Shop,Burger Joint,Grocery Store,Gym,Bank,Frozen Yogurt Shop,Japanese Restaurant,Farmers Market,Park,Pharmacy
